# This Script Scarpes Data From the Website https://periodictable.com/

In [1]:
import requests
from bs4 import BeautifulSoup

isotopes = []
for i in range(1, 119):
  # base URL for the isotopes
  base_url = "https://periodictable.com/Elements/{:03d}/data.html".format(i)

  r = requests.get(base_url)
  r.raise_for_status()
  soup = BeautifulSoup(r.text, 'html.parser')

  isotope_row = soup.find('a', string='Known Isotopes').find_parent('tr')
  isotope_table = isotope_row.find('table')
  isotope_links = isotope_table.find_all('a')
  isotope_urls = ['https://periodictable.com/' + link['href'][6:] for link in isotope_links]

  for url in isotope_urls:
    isotopes.append(url)


In [2]:
isotopes

['https://periodictable.com/Isotopes/001.1/index.html',
 'https://periodictable.com/Isotopes/001.2/index.html',
 'https://periodictable.com/Isotopes/001.3/index.html',
 'https://periodictable.com/Isotopes/001.4/index.html',
 'https://periodictable.com/Isotopes/001.5/index.html',
 'https://periodictable.com/Isotopes/001.6/index.html',
 'https://periodictable.com/Isotopes/001.7/index.html',
 'https://periodictable.com/Isotopes/002.2/index.html',
 'https://periodictable.com/Isotopes/002.3/index.html',
 'https://periodictable.com/Isotopes/002.4/index.html',
 'https://periodictable.com/Isotopes/002.5/index.html',
 'https://periodictable.com/Isotopes/002.6/index.html',
 'https://periodictable.com/Isotopes/002.7/index.html',
 'https://periodictable.com/Isotopes/002.8/index.html',
 'https://periodictable.com/Isotopes/002.9/index.html',
 'https://periodictable.com/Isotopes/002.10/index.html',
 'https://periodictable.com/Isotopes/003.3/index.html',
 'https://periodictable.com/Isotopes/003.4/inde

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def extract_data(soup, data_field):
    table_data_elements = soup.select('table')[7].find_all('td')
    for i, td in enumerate(table_data_elements):
      if td.text.replace("\xa0", " ") == data_field:
          return td.find_next_sibling().get_text()
    return None

def get_isotope_data(url):
    url = url[:-5] + '.dm.html' # Click 'Show Decay Models'
    r = requests.get(url)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, 'html.parser')


    p, n = [int(i[:-1]) for i in soup.select('table')[7].find_all('td')[4].text.split(' ')[1:]]
    half_life = soup.select('table')[7].find_all('td')[6].text
    atomic_weight = extract_data(soup, 'Atomic Weight')
    mass_excess = extract_data(soup, 'Mass Excess')
    binding_energy = extract_data(soup, 'Binding Energy')
    #decay_mode = soup.select('table')[10].find_all('td')[6].text

    return {'Protons': p,
            'Neutrons': n,
            'Atomic Weight': atomic_weight,
            'Mass Excess': mass_excess,
            'Binding Energy': binding_energy,
            #'Decay Mode': decay_mode,
            'Half-life': half_life}

get_isotope_data(isotopes[10])

df = pd.DataFrame([get_isotope_data(url) for url in isotopes])
#df.loc[df['Decay Mode'].str.contains('Binding'), 'Decay Mode'] = 'Stable'



In [5]:
df.to_csv('nuclear_physics_df_web.csv')